In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import re

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =

                title = html.xpath('//div[@class="title"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                list_text = [text.strip() for text in html.xpath('//div[@class="module-editor"]/descendant::*/text()')]
                while list_text[0] == '':
                    list_text.pop(0)
                while list_text[-1] == '':
                    list_text.pop(-1)

                description = '\n'.join(list_text)
                
                # = = = = = = = = = = = = = = =       
                
                list_description = []
                i = 0
                j = 1
                text_temp = ''
                while i < len(list_text):
                    if list_text[i].startswith(str(j) + '.'):
                        text_temp = ''
                        text_temp += list_text[i]
                        i += 1
                    elif list_text[i].startswith(str(j+1) + '.'):
                        list_description.append(text_temp)
                        j += 1
                    else:
                        text_temp += list_text[i]
                        i += 1

                list_description.append(text_temp)
    
                # = = = = = = = = = = = = = = =

                list_src = [src.strip() for src in html.xpath('//div[@class="swiper-slide"]/img/@src')]

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Title': title,
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Src': list_src[0],
                                         'Description': description}])

                for description in list_description:
                    field = re.match(r'^[0-9]+\..*?:', description)
                    if field == None:
                        df_temp.loc[0, description] = 'y'
                    else:
                        df_temp.loc[0, '.'.join(field.group().split('.')[1:]).split(':')[0].strip()] = description.replace(field.group(), '').strip()

                for i in range(1, len(list_src)):
                    df_temp.loc[0, 'Src_' + str(i)] = list_src[i]

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：642

[ok] - https://www.jcsportlinepro.com/ee-lc7016-carbon-fiber-rear-trunk-spoiler-for-zeekr-001-21-23.html
[尝试次数：1] - [剩余数量：622] - [当前时间：17:03:35]

[ok] - https://www.jcsportlinepro.com/ml-lq272-my-dry-carbon-fiber-rear-trunk-spoiler-for-bwm-3-series-f30-f35-f80-12-19.html
[尝试次数：1] - [剩余数量：621] - [当前时间：17:03:35]

[ok] - https://www.jcsportlinepro.com/ee-zdh7024-pro-dry-carbon-fiber-rear-spoiler-wing-for-tesla-model-3-base-sedan-2023.html
[尝试次数：1] - [剩余数量：620] - [当前时间：17:03:35]

[ok] - https://www.jcsportlinepro.com/ml-lq290-my-dry-carbon-fibre-g06-x6-rear-spoiler-for-bmw-f96-m6-x6-g06-m50i-sport-utility-4-door-2020-2023.html
[尝试次数：1] - [剩余数量：619] - [当前时间：17:03:36]

[ok] - https://www.jcsportlinepro.com/ee-wzy7013-carbon-fiber-rear-roof-spoiler-for-zeekr-001-21-23.html
[尝试次数：1] - [剩余数量：618] - [当前时间：17:03:36]

[ok] - https://www.jcsportlinepro.com/jc-hd571-real-carbon-fiber-rear-wing-trunk-spoiler-for-bmw-g87-m2-coupe-2023.html
[尝试次数：1] - [剩余数量：617] - [当前时间：17:03:36]

[ok] - https

[ok] - https://www.jcsportlinepro.com/ml-hax090-real-carbon-fiber-roof-wing-spoiler-for-kia-stinger.html
[尝试次数：1] - [剩余数量：573] - [当前时间：17:03:44]

[ok] - https://www.jcsportlinepro.com/ml-hax086-carbon-fiber-auto-x3-x3-m-rear-middle-spoiler-for-bmw-x3-x3-m-ix3-2018-2021.html
[尝试次数：1] - [剩余数量：572] - [当前时间：17:03:44]

[ok] - https://www.jcsportlinepro.com/ml-hjw017-real-carbon-fiber-rear-roof-spoiler-wing-for-bmw-ix3-x3-g01-mtech-g08-2019-2022.html
[尝试次数：1] - [剩余数量：571] - [当前时间：17:03:44]

[ok] - https://www.jcsportlinepro.com/ml-wzy013-carbon-fiber-rear-spoiler-for-2015-2020-mercedes-benz-s-coupe-w217-c217-s63-coupe.html
[尝试次数：1] - [剩余数量：570] - [当前时间：17:03:44]

[ok] - https://www.jcsportlinepro.com/ml-wjm088-my-dry-carbon-fiber-wing-spoiler-for-tesla-model-3-performance-sedan-4-door-2019-2023.html
[尝试次数：1] - [剩余数量：569] - [当前时间：17:03:45]

[ok] - https://www.jcsportlinepro.com/ml-pzj104-real-carbon-fiber-rear-trunk-lid-spoiler-boot-wing-lip-for-kia-stinger-2018-2023.html
[尝试次数：1] - [剩余数量：568

[ok] - https://www.jcsportlinepro.com/ed-hax9022-carbon-fiber-rear-trunk-wing-spoiler-for-porsche-911-991-turbo-s-2-door-2014-2016.html
[尝试次数：3] - [剩余数量：524] - [当前时间：17:03:55]

[ok] - https://www.jcsportlinepro.com/jc0127-glossy-matte-dry-carbon-fiber-g20-rear-boot-spoiler-wing-lip-for-bmw-3-series-g20-g28-330i-m340i-sedan-4-door-201.html
[尝试次数：1] - [剩余数量：523] - [当前时间：17:03:55]

[ok] - https://www.jcsportlinepro.com/ml-ybx200-for-bmw-1-series-f40-118i-m-sport-m135i-xdrive-hatchback-2020up-carbon-fiber-rear-roof-spoiler-window-wing-lip-1.html
[尝试次数：6] - [剩余数量：522] - [当前时间：17:03:55]

[ok] - https://www.jcsportlinepro.com/jc0030-glossy-matte-dry-carbon-fiber-g20-rear-boot-spoiler-wing-lip-for-bmw-3-series-g20-g28-330i-m340i-sedan-4-door-201.html
[尝试次数：2] - [剩余数量：521] - [当前时间：17:03:55]

[ok] - https://www.jcsportlinepro.com/ml-wjm022-my-carbon-fiber-rear-spoiler-for-bmw-8-series-g16-840i-gran-coupe-sedan-4-door-2019-2021.html
[尝试次数：6] - [剩余数量：520] - [当前时间：17:03:56]

[ok] - https://www.jcsp

[ok] - https://www.jcsportlinepro.com/ml-ybx122-carbon-fiber-a-class-w177-a250-rear-wing-spoiler-for-mercedes-benz-a220-hatchback-4-door-19-20.html
[尝试次数：2] - [剩余数量：480] - [当前时间：17:04:09]

[ok] - https://www.jcsportlinepro.com/ml-xm056-l-carbon-w205-spoiler-for-mercedes-base-sedan-4-door-c63-amg-2016.html
[尝试次数：1] - [剩余数量：479] - [当前时间：17:04:09]

[ok] - https://www.jcsportlinepro.com/ml-lq135-carbon-fibre-g16-m8-ducktail-spoiler-for-bmw-840i-gran-coupe-f93-m8-competition-sedan-4-door-2020-2021.html
[尝试次数：4] - [剩余数量：478] - [当前时间：17:04:09]

[ok] - https://www.jcsportlinepro.com/jc-adr8wy01-pro-dry-carbon-fiber-r8-ducktail-wing-spoiler-for-audi-r8-gen-2-v10-coupe-2016-2018.html
[尝试次数：3] - [剩余数量：477] - [当前时间：17:04:09]

[ok] - https://www.jcsportlinepro.com/ml-lq141-carbon-fiber-t6-rear-roof-spoiler-for-volkswagen-transporter-t6-van-2015-2020.html
[尝试次数：5] - [剩余数量：476] - [当前时间：17:04:10]

[ok] - https://www.jcsportlinepro.com/jc-zyy199-carbon-fiber-rear-spoiler-wing-for-mercedes-benz-cla-clas

[ok] - https://www.jcsportlinepro.com/ml-ybx090-carbon-fiber-rear-spoiler-for-audi-a7-s-line-s7-rs7-4-door-2019-2020.html
[尝试次数：16] - [剩余数量：434] - [当前时间：17:04:29]

[ok] - https://www.jcsportlinepro.com/jc-wsm370-carbon-fiber-rear-trunk-spoiler-highkick-wing-fit-for-lamborghini-huracan-2014-2018.html
[尝试次数：1] - [剩余数量：433] - [当前时间：17:04:29]

[ok] - https://www.jcsportlinepro.com/jc-zyy341-carbon-fiber-rear-trunk-spoiler-wing-for-toyota-86-brz-coupe-2013-2019.html
[尝试次数：46] - [剩余数量：432] - [当前时间：17:04:30]

[ok] - https://www.jcsportlinepro.com/jc-wsm339-carbon-fiber-refit-rear-trunk-spoiler-lid-wing-fit-for-toyota-gt86-brz-2013-2020.html
[尝试次数：5] - [剩余数量：431] - [当前时间：17:04:30]

[ok] - https://www.jcsportlinepro.com/jc-wsm376-carbon-fiber-rear-wing-trunk-spoiler-for-honda-civic-10th-sedan-4-door.html
[尝试次数：3] - [剩余数量：430] - [当前时间：17:04:30]

[ok] - https://www.jcsportlinepro.com/jc-sm003-carbon-fiber-rear-trunk-spoiler-boot-lip-wing-spoiler-for-ford-mustang-coupe-2015-2018.html
[尝试次数：10] - [

...

[ok] - https://www.jcsportlinepro.com/jc-hly241-carbon-fiber-e36-car-rear-spoiler-for-bmw-318i-320i-325i-328i-m3-sedan-4-door-1991-1998.html
[尝试次数：8] - [剩余数量：193] - [当前时间：17:05:54]

[ok] - https://www.jcsportlinepro.com/jc-hly202-g30-carbon-fiber-rear-wing-spoiler-for-bmw-g31-g38-520i-530i-540i-m-sport-17-19.html
[尝试次数：1] - [剩余数量：192] - [当前时间：17:05:55]

[ok] - https://www.jcsportlinepro.com/jc-hly240-m4-style-carbon-fiber-e36-trunk-spoiler-for-bmw-318i-320i-325i-328i-m3-sedan-4-door-91-98.html
[尝试次数：4] - [剩余数量：191] - [当前时间：17:05:55]

[ok] - https://www.jcsportlinepro.com/jc-hly157-carbon-fiber-rear-spoiler-wing-for-bmw-e46-3-series-4-door-sedan-m-style-window-roof-1996-2005.html
[尝试次数：15] - [剩余数量：190] - [当前时间：17:05:55]

[ok] - https://www.jcsportlinepro.com/jc-hly371.html
[尝试次数：3] - [剩余数量：189] - [当前时间：17:05:56]

[ok] - https://www.jcsportlinepro.com/jc-hly203-carbon-fiber-f10-m5-ducktail-spoiler-for-bmw-520i-528i-530i-535i-540i-m-sport-sedan-2011-2017.html
[尝试次数：3] - [剩余数量：188] - [当前时间

[ok] - https://www.jcsportlinepro.com/jc-xp589-for-mercedes-benz-c-class-w205-c180-c200-c250-c300-c350-c450-4-door-2015-2016-rear-spoiler-trunk-boot-wings-car.html
[尝试次数：16] - [剩余数量：147] - [当前时间：17:06:13]

[ok] - https://www.jcsportlinepro.com/jc-zyy210-carbon-fiber-w213-rear-trunk-lid-spoiler-for-mercedes-benz-e200-e350-e63-amg-sedan-16-19.html
[尝试次数：20] - [剩余数量：146] - [当前时间：17:06:13]

[ok] - https://www.jcsportlinepro.com/jc-hd056-carbon-fiber-car-rear-roof-spoiler-window-wing-lip-for-audi-a3-hatchback-2014-2015-2016.html
[尝试次数：7] - [剩余数量：145] - [当前时间：17:06:14]

[ok] - https://www.jcsportlinepro.com/jc-aua4b8030-carbon-fiber-car-rear-trunk-spoiler-for-audi-a4-b8-b9-2009-2016-boot-spoiler-trunk-wing-lip.html
[尝试次数：1] - [剩余数量：144] - [当前时间：17:06:14]

[ok] - https://www.jcsportlinepro.com/ml-ybx029-carbon-fiber-rear-trunk-spoiler-for-dodge-challenger-srt-hellcat-coupe-2-door-15-18.html
[尝试次数：46] - [剩余数量：143] - [当前时间：17:06:14]

[ok] - https://www.jcsportlinepro.com/ml-xm241-carbon-fiber-r

[ok] - https://www.jcsportlinepro.com/jc-bme920527-car-styling-carbon-fiber-roof-window-spoiler-auto-wing-lip-for-bmw-325i-e92-coupe-2007-2013.html
[尝试次数：9] - [剩余数量：101] - [当前时间：17:06:30]

[ok] - https://www.jcsportlinepro.com/jc-bme900507-carbon-fiber-car-rear-roof-spoiler-lip-wing-for-bmw-3-series-e90-2005-2008.html
[尝试次数：3] - [剩余数量：100] - [当前时间：17:06:30]

[ok] - https://www.jcsportlinepro.com/jc-bme920545-carbon-fiber-rear-trunk-spoiler-for-bmw-3-series-e92-coupe-m3-m-sport-2door-2006-2012-car-rear-boot-wing-li.html
[尝试次数：10] - [剩余数量：99] - [当前时间：17:06:30]

[ok] - https://www.jcsportlinepro.com/jc-bmef3004-carbon-fiber-rear-spoiler-trunk-boot-duck-wings-for-bmw-f30-316i-318i-320i-328i-335i-326d-f80-m3-all-3-serie.html
[尝试次数：1] - [剩余数量：98] - [当前时间：17:06:30]

[ok] - https://www.jcsportlinepro.com/jc-bme900510-car-styling-carbon-fiber-auto-automotive-rear-spoiler-wing-for-bmw-3-series-e90-m3-2005-2012.html
[尝试次数：1] - [剩余数量：97] - [当前时间：17:06:30]

[ok] - https://www.jcsportlinepro.com/jc-

[ok] - https://www.jcsportlinepro.com/ml-xm073-carbon-fiber-rear-roof-spoiler-for-volkswagen-golf-7-vii-mk7-gti-2014-2017.html
[尝试次数：8] - [剩余数量：53] - [当前时间：17:06:45]

[ok] - https://www.jcsportlinepro.com/jc-xp238-carbon-fiber-car-spoiler-for-volkswagen-vw-golf-6-vi-mk6.html
[尝试次数：5] - [剩余数量：52] - [当前时间：17:06:45]

[ok] - https://www.jcsportlinepro.com/jc-vwsc0613-car-carbon-fiber-rear-window-spoiler-wing-for-vw-scirocco.html
[尝试次数：12] - [剩余数量：51] - [当前时间：17:06:45]

[ok] - https://www.jcsportlinepro.com/jc-hd156-carbon-fiber-wing-roof-spoiler-for-smart-fortwo-coupe-2-door-16-17.html
[尝试次数：4] - [剩余数量：50] - [当前时间：17:06:45]

[ok] - https://www.jcsportlinepro.com/ml-xm284-carbon-fiber-roof-spoiler-for-mercedes-benz-e-class-e200-e400-e500-coupe-2017-2018.html
[尝试次数：3] - [剩余数量：49] - [当前时间：17:06:46]

[ok] - https://www.jcsportlinepro.com/ml-xm299-carbon-fiber-rear-trunk-spoiler-for-mercedes-c238-e200-e400-e500-coupe-2-door-17-18.html
[尝试次数：1] - [剩余数量：48] - [当前时间：17:06:46]

[ok] - https://www.j

[ok] - https://www.jcsportlinepro.com/jc-hly360-carbon-fiber-rear-roof-wing-spoiler-for-bmw-3-series-g20-g21-g28-2019-2020.html
[尝试次数：1] - [剩余数量：4] - [当前时间：17:07:01]

[ok] - https://www.jcsportlinepro.com/ml-qmn001-carbon-fiber-rear-middle-spoiler-for-mercedes-benz-v-class-v200d-v250-2018-2019.html
[尝试次数：2] - [剩余数量：3] - [当前时间：17:07:02]

[ok] - https://www.jcsportlinepro.com/jc-hly215-carbon-fiber-s6-car-trunk-spoiler-for-audi-a6-c6-4-door-05-11.html
[尝试次数：1] - [剩余数量：2] - [当前时间：17:07:02]

[ok] - https://www.jcsportlinepro.com/jc-hd062-carbon-fiber-roof-spoiler-for-mecedes-ben-z-w205-2015-2016.html
[尝试次数：6] - [剩余数量：1] - [当前时间：17:07:03]

[ok] - https://www.jcsportlinepro.com/jc-hly352-carbon-fiber-rear-wing-spoiler-for-bmw-g20-330i-m340i-g20-sedan-2019-2020.html
[尝试次数：2] - [剩余数量：0] - [当前时间：17:07:03]

[ok] - https://www.jcsportlinepro.com/ml-lq011-carbon-fiber-roof-spoiler-for-mercedes-benz-v-glass-v250-2018-2019.html
[尝试次数：1] - [剩余数量：0] - [当前时间：17:07:03]

[ok] - https://www.jcsportlinepro